In [1]:
import requests
import json
import time

# get api key from Leap AI https://www.tryleap.ai
API_KEY = "de94877e-d732-447c-857b-de9cff3a95f2"

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

In [2]:
def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 100,
        "width": 128,
        "height": 128,
        "numberOfImages": 4,
        "promptStrength": 8,
        "enhancePrompt": False,
        "restoreFaces": True,
        "upscaleBy": "x1"
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    print(data)
    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status


def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    images = None

    if len(data["images"]):
        images = data["images"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, images




In [6]:
# modelID refers to Realistic Vision model. We can now use it to generate images from.
# For more models go here: https://docs.tryleap.ai/reference/pre-trained-models
model_id = "eab32df0-de26-4b83-a908-a83f3015e971"
inference_id, status = generate_image(
    model_id,
    prompt="A profile photo of Superman in Metropolis, wearing a dark style caped suit, \
    at a high rise resident terrace on a cool evening sunset background, \
    just showing above torso, generated by anime style"
)
while status != "finished":
    time.sleep(10)
    inference_id, status, images = get_inference_job(model_id, inference_id)

for image in images:
    print("Image ready:", image["uri"])

{'status': 'queued', 'id': '2c2b9e3c-d23e-4153-9c8d-a92921a511b8', 'prompt': 'A profile photo of Superman in Metropolis, wearing a dark style caped suit, at a high rise resident terrace on a cool evening sunset background, just showing above torso, generated by anime style', 'negativePrompt': '', 'seed': 760585644, 'width': 512, 'height': 512, 'numberOfImages': 4, 'steps': 50, 'createdAt': '2023-06-24T15:36:06.324Z', 'promptStrength': 7, 'images': [], 'modelId': 'eab32df0-de26-4b83-a908-a83f3015e971', 'upscalingOption': 'x1', 'sampler': 'euler_a'}
Inference ID: 2c2b9e3c-d23e-4153-9c8d-a92921a511b8. Status: queued
Inference ID: 2c2b9e3c-d23e-4153-9c8d-a92921a511b8. State: processing
Inference ID: 2c2b9e3c-d23e-4153-9c8d-a92921a511b8. State: processing
Inference ID: 2c2b9e3c-d23e-4153-9c8d-a92921a511b8. State: finished
Image ready: https://static.tryleap.ai/image-gen-2c2b9e3c-d23e-4153-9c8d-a92921a511b8/generated_images/1.png
Image ready: https://static.tryleap.ai/image-gen-2c2b9e3c-d23e